In [14]:
import pandas as pd
import datetime as dt
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn'

# Consigna

Vamos a trabajar con una base de datos elaborada por [Sysarmy](https://sysarmy.com/es/) (una comunidad de profesionales en el área de sistemas) que contiene los resultados de una encuesta se suedlos realizada en 2020. La encuesta tiene algunso campos tipificados y otros campos de texto libre.

Queremos analizar los resultados de esta encuesta. Nos interesan especialmente aquellas respuestas vinculadas a profecionales que trabajan con datos y que poseen título universitario (para eliminar algunos casos con comportamiento atípico). Los objetivos de esta práctica serán:

1. Construir una variable que nos indique hace cuánto tiempo fue respondida la encuesta.

2. Encontrar el salario promedio para los trabajos vinculados a datos. Puede ser general o abierto por las distintas variantes.

3. Contabilizar cuantos lenguajes de programación maneja la persona.


In [15]:
sueldos = pd.read_csv("https://raw.githubusercontent.com/eea-uba/EEA-2020/master/Fuentes/encuesta_sueldos_sysarmy_1s2020.csv")
sueldos.head()


,timestamp,me_identifico,tengo,estoy_trabajando_en,donde_estas_trabajando,anos_de_experiencia,anos_en_la_empresa_actual,anos_en_el_puesto_actual,gente_a_cargo,nivel_de_estudios_alcanzado,...,sos_miembro_de_alguna_comunidad_it,cantidad_de_empleados,actividad_principal,la_recomendas_como_un_buen_lugar_para_trabajar,como_calificas_las_politicas_de_diversidad_e_inclusion,a_cuantos_kilometros_de_tu_casa_queda_la_oficina,beneficios_extra,habias_respondido_nuestra_encuesta_en_ediciones_anteriores,cuales_consideras_que_son_las_mejores_empresas_de_it_para_trabajar_en_este_momentoen_tu_ciudad,sueldo_dolarizado
0,1/31/2020 6:50:26,Hombre,32,Argentina,Ciudad Autónoma de Buenos Aires,10,3,1,0,Secundario,...,NaN,10001+,Producto basado en Software,8,10,6,"Capacitaciones y/o cursos, Comidas pagas / sub...",No,cognizant,False
1,1/28/2020 9:27:48,Hombre,30,Argentina,Ciudad Autónoma de Buenos Aires,3,3,3,0,Terciario,...,NaN,51-100,Otras industrias,7,3,35,Capacitaciones y/o cursos,No,mercadolibre,False
2,1/29/2020 16:54:29,Mujer,40,Argentina,Ciudad Autónoma de Buenos Aires,15,3,3,1,Universitario,...,NaN,11-50,Servicios / Consultoría de Software / Digital,8,8,30,"Abono de celular y/o Internet, Capacitaciones ...",No,NaN,False
3,2/1/2020 5:46:25,Mujer,36,Argentina,Ciudad Autónoma de Buenos Aires,10,3,3,0,Universitario,...,NaN,51-100,Otras industrias,7,5,4,Viáticos,No,NaN,False
4,2/1/2020 17:51:21,Hombre,29,Argentina,Ciudad Autónoma de Buenos Aires,10,2,2,0,Universitario,...,NaN,11-50,Otras industrias,5,8,7,Ninguna de las anteriores,No,NaN,False


In [16]:
# observamos faltantes y tipos de datos
sueldos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5982 entries, 0 to 5981
Data columns (total 53 columns):
 #   Column                                                                                          Non-Null Count  Dtype  
---  ------                                                                                          --------------  -----  
 0   timestamp                                                                                       5982 non-null   object 
 1   me_identifico                                                                                   5982 non-null   object 
 2   tengo                                                                                           5982 non-null   int64  
 3   estoy_trabajando_en                                                                             5982 non-null   object 
 4   donde_estas_trabajando                                                                          5982 non-null   object 
 5   anos_de_exper

In [17]:
# Antes de comenzar, filtramos solo los registros con educacion universitaria
condicion = ~(sueldos['nivel_de_estudios_alcanzado'].str.lower().isin(
    ['secundario', 'primario']))
sueldos = sueldos[condicion]
sueldos.head()

,timestamp,me_identifico,tengo,estoy_trabajando_en,donde_estas_trabajando,anos_de_experiencia,anos_en_la_empresa_actual,anos_en_el_puesto_actual,gente_a_cargo,nivel_de_estudios_alcanzado,...,sos_miembro_de_alguna_comunidad_it,cantidad_de_empleados,actividad_principal,la_recomendas_como_un_buen_lugar_para_trabajar,como_calificas_las_politicas_de_diversidad_e_inclusion,a_cuantos_kilometros_de_tu_casa_queda_la_oficina,beneficios_extra,habias_respondido_nuestra_encuesta_en_ediciones_anteriores,cuales_consideras_que_son_las_mejores_empresas_de_it_para_trabajar_en_este_momentoen_tu_ciudad,sueldo_dolarizado
1,1/28/2020 9:27:48,Hombre,30,Argentina,Ciudad Autónoma de Buenos Aires,3,3,3,0,Terciario,...,NaN,51-100,Otras industrias,7,3,35,Capacitaciones y/o cursos,No,mercadolibre,False
2,1/29/2020 16:54:29,Mujer,40,Argentina,Ciudad Autónoma de Buenos Aires,15,3,3,1,Universitario,...,NaN,11-50,Servicios / Consultoría de Software / Digital,8,8,30,"Abono de celular y/o Internet, Capacitaciones ...",No,NaN,False
3,2/1/2020 5:46:25,Mujer,36,Argentina,Ciudad Autónoma de Buenos Aires,10,3,3,0,Universitario,...,NaN,51-100,Otras industrias,7,5,4,Viáticos,No,NaN,False
4,2/1/2020 17:51:21,Hombre,29,Argentina,Ciudad Autónoma de Buenos Aires,10,2,2,0,Universitario,...,NaN,11-50,Otras industrias,5,8,7,Ninguna de las anteriores,No,NaN,False
5,2/1/2020 23:13:41,Hombre,27,Argentina,Ciudad Autónoma de Buenos Aires,4,3,2,0,Universitario,...,No,1-10,Otras industrias,2,6,12,"Capacitaciones y/o cursos, Descuentos varios (...",No,NaN,False


## 1) Fecha de encuesta

Pista: la función **datetime.datetime.now()** nos devuelve fecha y hora actuales.

In [5]:
sueldos.timestamp.unique()

array(['1/28/2020 9:27:48', '1/29/2020 16:54:29', '2/1/2020 5:46:25', ...,
       '1/31/2020 13:54:08', '1/31/2020 19:19:38', '2/2/2020 4:45:31'],
      dtype=object)

In [19]:
# convertimos la variable datestamp a date

algo = dt.datetime.strptime(sueldos['timestamp'].str, '%m/%d/%Y %H:%M:%S')
algo
#sueldos['timestamp'] = pd.to_datetime(sueldos['timestamp'],format='mixed')


TypeError: strptime() argument 1 must be str, not StringMethods

In [7]:
sueldos["timestamp"]

1      2020-01-28 09:27:48
2      2020-01-29 16:54:29
3      2020-02-01 05:46:25
4      2020-02-01 17:51:21
5      2020-02-01 23:13:41
               ...        
5977   2020-01-30 04:48:47
5978   2020-01-30 11:41:15
5979   2020-01-31 13:54:08
5980   2020-01-31 19:19:38
5981   2020-02-02 04:45:31
Name: timestamp, Length: 5425, dtype: datetime64[ns]

In [12]:
# generamos la variable de antiguedad (diferencia con hoy)
today = dt.datetime.now()



## 2) Trabajo con datos

In [ ]:
sueldos.trabajo_de.unique()

In [ ]:
# llevamos todo a minuscula


In [ ]:
# creo una variable binaria para ver si trabaja con datos


In [ ]:
# Verificamos nuestro filtro


In [ ]:
# Filtramos los profesionales de datos
sueldos_data = 

In [ ]:
# Normalizamos el puesto de 'data entry' y 'data analyst


In [ ]:
#Vemos los valores unicos


In [ ]:
# Ahora vemos el salario promedio


In [ ]:
# Agrupamos por "trabajo_de" obteniendo el promedio y la mediana 
sueldos_data.groupby('trabajo_de').agg({'salario_mensual_bruto_en_tu_moneda_local': [np.mean, np.median]})

## 3) Lenguajes de programación

In [ ]:
#Observamos la variable lenguajes_de_programacion

In [ ]:
#La transformamos en una lista con split


In [ ]:
sueldos_data.head()

In [ ]:
#Creamos una variable cantidad_lenguajes  


In [ ]:
#Y volvemos a obtener el promedio y la mediana
sueldos_data.groupby('trabajo_de').agg({'cantidad_lenguajes': [np.mean, np.median]                                })